<a href="https://colab.research.google.com/github/midhunjmes/midhun-26-02-24/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install openpyxl
!pip install presidio_analyzer
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 87.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can

In [2]:
!pip install presidio_anonymizer

In [2]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.4 MB/s eta 0:00:00


In [2]:
# from presidio_analyzer import AnalyzerEngine

# analyzer = AnalyzerEngine()
# for recognizer in analyzer.get_recognizers():
#     print(recognizer.supported_entities)

In [34]:

from faker import Faker
import pandas as pd
import spacy
import re
import openpyxl
import json
from presidio_analyzer import AnalyzerEngine
analyzer = AnalyzerEngine()
mapping={}
nlp = spacy.load("en_core_web_sm")
fake=Faker()

#-------------------------------------------------------------------------------------------------------------------------------
#function to find out the noun values dominating columns considering it will be a sensitive data if there is so many nouns
#------------------------------------------------------------------------------------------------------------------------------
def detect_noun(file_path):
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, engine="python")
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    sensitive = []

    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:
            continue  # Skip purely numeric columns

        text_samples = df[col].astype(str).head(5)  # Take first 5 values
        noun_count = 0
        total_count = 0

        for value in text_samples:
            if "%" in value or value.replace(".", "").isdigit():
                continue  # Skip percentage or number-like values

            doc = nlp(value)
            for token in doc:
                if token.pos_ in ['NOUN', 'PROPN']:
                    noun_count += 1
                total_count += 1

        # Mark column as sensitive only if a significant portion are nouns
        if total_count > 0 and (noun_count / total_count) > 0.2:
            sensitive.append(col)

    return sensitive


#------------------------------------------------------------------------------------------
#finding out the descriptive data columns which may have sensitive data in the form of text
#------------------------------------------------------------------------------------------
def descriptive_columns(file_path):
    # Define keywords to filter out
    keywords = ["description", "remarks", "notes", "comments", "observations", "details", "summary", "explanation",
    "reviews", "feedback", "testimonials", "opinions", "assessment", "suggestions", "experience",
    "incident_report", "case_notes", "audit_notes", "findings", "status_update", "history", "progress_report",
    "additional_info", "clarifications", "justification", "annotations", "excerpts", "statement", "explanation_text"]

    # Ensure columns are properly loaded from CSV/Excel
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, nrows=1)  # Read only header
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path, nrows=1, engine="openpyxl")  # Read only header
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    # Get actual column names
    all_columns = df.columns.tolist()


    des=[col for col in all_columns if any(re.search(keyword, col, re.IGNORECASE) for keyword in keywords)]
    return des

#-----------------------------------------------------------------
#anonymizing descriptive values
#-----------------------------------------------------------------
def detect_noun_desc(text):
    doc = nlp(text)
    modified_text=[]
    for token in doc:
        if token.pos_ in ['PROPN']:
            modified_text.append("<sensitive>")
        else:
            modified_text.append(token.text)
    text=" ".join(modified_text)
    return text


#------------------------------------------------------------------
#function for detecting numerical sensitive
#------------------------------------------------------------------
def detect_sensitive_numerical(file_path,sensitive):
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, engine="python")
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or Excel file.")

    sensitive_columns = []

    for col in df.columns:
        if col not in sensitive:
          text_samples = df[col].astype(str).head(5)  # Convert first 5 values to string

          for value in text_samples:
              if pd.notna(value) and value.strip():
                  results = analyzer.analyze(text=value, language="en")

                  for result in results:
                      # print(result,result.entity_type)
                      if result.entity_type in ["PHONE_NUMBER", "CREDIT_CARD", "IBAN", "US_SSN","EMAIL"]:
                          sensitive_columns.append(col)
                          break  # If any value in the column is sensitive, mark the whole column

    return list(set(sensitive_columns))
#-------------------------------------------------------------------
#function to anonymize a excel file
#-------------------------------------------------------------------
def excel_an(input_file, output_file):
    df = pd.read_excel(input_file)  # Read as string for safety

    sensitive_old = detect_noun(input_file)
    desc = descriptive_columns(input_file)
    sensitive = list(set(sensitive_old) - set(desc))
    num_sensitive=detect_sensitive_numerical(input_file,sensitive_old)
    sensitive=list(set(sensitive)+set(num_sensitive))

    column_counters = {col: 1 for col in df.columns if col in sensitive_old}
    column_mappings = {col: {} for col in sensitive}  # Store mappings for each column
    mapping = {}

    # Anonymize sensitive columns while maintaining consistency
    for col in sensitive:
        new_values = []
        for val in df[col].astype(str):
            if pd.notna(val):
                if val in column_mappings[col]:
                    anonymized_value = column_mappings[col][val]  # Use existing mapping
                else:
                    anonymized_value = f"{col}{column_counters[col]}"
                    column_mappings[col][val] = anonymized_value  # Store new mapping
                    column_counters[col] += 1  # Increment counter

                mapping[anonymized_value] = val
                new_values.append(anonymized_value)
            else:
                new_values.append(val)

        df[col] = new_values

    # Anonymize descriptive columns
    for col in desc:
        for idx, val in enumerate(df[col].astype(str)):
            if pd.notna(val):
                an_values = detect_noun_desc(val)
                df.at[idx, col] = an_values
                mapping[an_values] = val

    df.to_excel(output_file, index=False, sheet_name="Anonymized Data")
    print(f"✅ Anonymized file saved as {output_file}")

    # Save mapping as JSON
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)

#------------------------------------------------------
#function for de-anonymizing excel data
#------------------------------------------------------

def excel_dean(input_file, output_file, mapping_file):
    print("🔄 Loading data...")

    with open(mapping_file, "r") as f:
        mapping = json.load(f)

    df = pd.read_excel(input_file)  # Read as string for safety
    mapping_keys = set(mapping.keys())
    df = df.applymap(lambda x: mapping[x] if x in mapping_keys else x)
    df.to_excel(output_file, index=False, sheet_name="De-anonymized Data")

    print(f"✅ De-anonymized file saved as {output_file}")

#----------------------------------------------------------
#function for anonymizing csv data
#----------------------------------------------------------
def csv_an(input_file, output_file):
    df = pd.read_csv(input_file, engine="python")

    sensitive_old = detect_noun(input_file)
    desc = descriptive_columns(input_file)
    sensitive = list(set(sensitive_old) - set(desc))
    num_sensitive=detect_sensitive_numerical(input_file,sensitive_old)
    sensitive=sensitive+num_sensitive
    print(sensitive)

    # column_counters = {col: 1 for col in df.columns if col in sensitive}
    # column_mappings = {col: {} for col in sensitive}  # Store mappings for each column

    # # Anonymize sensitive columns while maintaining consistency
    # for col in sensitive:
    #     new_values = []
    #     for val in df[col].astype(str):
    #         if pd.notna(val):
    #             if val in column_mappings[col]:
    #                 anonymized_value = column_mappings[col][val]  # Use existing mapping
    #             else:
    #                 anonymized_value = f"{col}{column_counters[col]}"
    #                 column_mappings[col][val] = anonymized_value  # Store new mapping
    #                 column_counters[col] += 1  # Increment counter

    #             mapping[anonymized_value] = val
    #             new_values.append(anonymized_value)
    #         else:
    #             new_values.append(val)

    #     df[col] = new_values


    column_mappings = {col: {} for col in sensitive}  # Store mappings for each column

    # Anonymize sensitive columns while maintaining consistency
    for col in sensitive:
        if col in df.columns:  # Ensure column exists
            # Convert to string only once for entire column
            str_col = df[col].astype(str)

            # Create mask for non-NA values
            mask = str_col.notna()

            # Apply faking_data only to non-NA values
            df[col] = str_col.where(~mask, str_col[mask].apply(anonymize_text))

        # Anonymize descriptive columns
    for col in desc:
        for idx, val in enumerate(df[col].astype(str)):
            if pd.notna(val):
                an_values = detect_noun_desc(val)
                df.at[idx, col] = an_values
                mapping[an_values] = val

    df.to_csv(output_file, index=False)
    print(f"✅ Anonymized file saved as {output_file}")

    # Save mapping as JSON
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)

#------------------------------------------------------------
#function for de anonymizing csv data
#------------------------------------------------------------
def csv_dean(input_file, output_file, mapping_file):
    print("🔄 Loading data...")

    with open(mapping_file, "r") as f:
        mapping = json.load(f)
    df = pd.read_csv(input_file, engine="python", dtype=str)  # Read as string for safety
    mapping_keys = set(mapping.keys())
    df = df.applymap(lambda x: mapping[x] if x in mapping_keys else x)
    df.to_csv(output_file, index=False)

    print(f"✅ De-anonymized file saved as {output_file}")
#------------------------------------------------------------
#function for faker
#------------------------------------------------------------
from faker import Faker
import re
fake=Faker()
PHONE_REGEX = r"(\+?\d{1,3}[-.\s]?)?\(?\d{1,4}\)?[-.\s]?\d{1,4}[-.\s]?\d{1,9}"
EMAIL_REGEX = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"
def anonymize_text(text):
    results = analyzer.analyze(text=text, language="en")

    new_text = text
    for result in sorted(results, key=lambda x: x.start, reverse=True):  # Process from the end to avoid shifting indexes
        fake_value = ""
        if result.entity_type == "PERSON":
            fake_value = fake.name()
        elif result.entity_type == "GPE":
            fake_value = fake.city()
        elif result.entity_type == "ORG":
            fake_value = fake.company()


        mapping[fake_value] = text[result.start:result.end]
        new_text = new_text[:result.start] + fake_value + new_text[result.end:]

    return new_text


#--------------------------------------------------------------
#function to determine whcih type of data is need to perform
#--------------------------------------------------------------
def anonymization(input_file):
    if input_file.endswith(".csv"):
        csv_an(input_file,"intermediate.csv")
        csv_dean("intermediate.csv","deanonymized.csv","mappings.json")
    elif input_file.endswith(".xlsx"):
        excel_an(input_file,"intermediate.xlsx")
        excel_dean("intermediate.xlsx","deanonymized.xlsx","mappings.json")

file="numerical.csv"
anonymization(file)

['Address', 'Name', 'Phone Number', 'Credit Card']
✅ Anonymized file saved as intermediate.csv
🔄 Loading data...
✅ De-anonymized file saved as deanonymized.csv


<ipython-input-34-e6def0f03943>:261: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: mapping[x] if x in mapping_keys else x)


In [25]:
!pip install faker

In [27]:
from faker import Faker
import re
fake=Faker()
PHONE_REGEX = r"(\+?\d{1,3}[-.\s]?)?\(?\d{1,4}\)?[-.\s]?\d{1,4}[-.\s]?\d{1,9}"
EMAIL_REGEX = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"
def faking_data(text):
  text=re.sub(PHONE_REGEX,fake.phone_number(),text)
  text=re.sub(EMAIL_REGEX,fake.email(),text)
  doc=nlp(text)
  new_text=text
  for ent in doc.ents:
    fake_value=""
    if ent.label_=="PERSON":
      fake_value=fake.name()
    elif ent.label_=="GPE":
      fake_value=fake.city()
    elif ent.label_=="ORG":
      fake_value=fake.company()
    if fake_value:
      new_text=new_text.replace(ent.text,fake_value)
  return new_text


In [ ]:
def csv_an(input_file, output_file):
    df = pd.read_csv(input_file, engine="python")

    sensitive_old = detect_noun(input_file)
    desc = descriptive_columns(input_file)
    sensitive = list(set(sensitive_old) - set(desc))
    num_sensitive=detect_sensitive_numerical(input_file,sensitive_old)
    sensitive=sensitive+num_sensitive
    # column_counters = {col: 1 for col in df.columns if col in sensitive}
    column_mappings = {}  # Store mappings for each column

    # Anonymize sensitive columns while maintaining consistency
    for col in sensitive:
        new_values = []
        for val in df[col].astype(str):
            if pd.notna(val):
                if val in column_mappings[col]:
                    anonymized_value = column_mappings[val]  # Use existing mapping
                else:
                    anonymized_value = faking_data(val)
                    column_mappings[val] = anonymized_value  # Store new mapping
                mapping[anonymized_value] = val
                new_values.append(anonymized_value)
            else:
                new_values.append(val)

        df[col] = new_values

    # Anonymize descriptive columns
    for col in desc:
        for idx, val in enumerate(df[col].astype(str)):
            if pd.notna(val):
                an_values = detect_noun_desc(val)
                df.at[idx, col] = an_values
                mapping[an_values] = val

    df.to_csv(output_file, index=False)
    print(f"✅ Anonymized file saved as {output_file}")

    # Save mapping as JSON
    with open("mappings.json", "w") as f:
        json.dump(mapping, f)

In [32]:
!pip install mimesis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 31.4 MB/s eta 0:00:00


In [35]:
print(dir(fake))

['__annotations__', '__class__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_factories', '_factory_map', '_locales', '_map_provider_method', '_optional_proxy', '_select_factory', '_select_factory_choice', '_select_factory_distribution', '_unique_proxy', '_weights', 'aba', 'add_provider', 'address', 'administrative_unit', 'am_pm', 'android_platform_token', 'ascii_company_email', 'ascii_email', 'ascii_free_email', 'ascii_safe_email', 'bank_country', 'basic_phone_number', 'bban', 'binary', 'boolean', 'bothify', 'bs', 'building_number', 'cache_pattern', 'catch_phrase', 'century', 'chrome', 'city', 'city_prefix', 'city_suffix', 'c

In [39]:
# Testing columnwise replacing using Faker
import os
import re
import time
import json
import pandas as pd
from faker import Faker
from presidio_analyzer import AnalyzerEngine

# Initialize Faker & Presidio Analyzer
fake = Faker()
Faker.seed(42)
analyzer = AnalyzerEngine()

# Mapping file for storing forward/reverse mappings
MAPPING_FILE = "mappings.json"
forward_mapping = {}
reverse_mapping = {}

def time_it(func):
    """Decorator to measure execution time."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\nExecution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper

def load_mapping():
    """Loads existing mappings from a file if available."""
    global forward_mapping, reverse_mapping
    if os.path.exists(MAPPING_FILE):
        try:
            with open(MAPPING_FILE, "r") as f:
                data = json.load(f)
                forward_mapping = data.get("forward_mapping", {})
                reverse_mapping = data.get("reverse_mapping", {})
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON file: {e}")
            forward_mapping = {}
            reverse_mapping = {}

def save_mapping():
    """Saves the mapping to a JSON file."""
    with open(MAPPING_FILE, "w") as f:
        json.dump({"forward_mapping": forward_mapping, "reverse_mapping": reverse_mapping}, f, indent=4)

@time_it
def analyze_column(df):
    """Detects sensitive columns using Presidio."""
    entity_columns = {}
    for col in df.columns:
        unique_values = df[col].dropna().astype(str).unique()[:15]
        entity_counts = {}

        for value in unique_values:
            results = analyzer.analyze(text=value, language="en")
            for result in results:
                entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

        if entity_counts:
            predominant_entity = max(entity_counts, key=entity_counts.get)
            if predominant_entity not in entity_columns:
                entity_columns[predominant_entity] = []
            entity_columns[predominant_entity].append(col)

    return entity_columns

def load_and_analyze(file_path):
    """Loads CSV/Excel and analyzes sensitive columns."""
    df = pd.read_csv(file_path) if file_path.endswith(".csv") else pd.read_excel(file_path)
    return analyze_column(df)

def generate_fake_values(df, entity_type, column):
    """Generates fake values for an entire column while preserving uniqueness."""
    faker_mapping = {
        "PERSON": fake.name,
        "FIRST_NAME": fake.first_name,
        "LAST_NAME": fake.last_name,
        "EMAIL": fake.email,
        "URL": fake.url,
        "PHONE_NUMBER": fake.phone_number,
        "CREDIT_CARD": fake.credit_card_number,
        "IBAN": fake.iban,
        "US_SSN": fake.ssn,
        "LOCATION": fake.company,
        "STREET_ADDRESS": fake.street_address,
        "CITY": fake.city,
        "STATE": fake.state,
        "COUNTRY": fake.country,
        "ZIP_CODE": fake.zipcode,
        "ORGANIZATION": fake.company,
        "JOB_TITLE": fake.job,
        "USERNAME": fake.user_name,
        "PASSWORD": fake.password,
        "IP_ADDRESS": fake.ipv4,
        "MAC_ADDRESS": fake.mac_address,
        "LICENSE_PLATE": fake.license_plate,
        "UUID": fake.uuid4,
        "BANK_ACCOUNT": fake.bban,
        "TRANSACTION_ID": fake.uuid4,
        "DEVICE_ID": fake.uuid4,
        "ID": fake.uuid4,
    }

    if entity_type not in faker_mapping:
        return df  # Skip if no Faker function exists

    faker_func = faker_mapping[entity_type]

    # Replace the entire column in one go with mapped fake values
    original_values = df[column].astype(str).dropna().unique()
    fake_values = [forward_mapping.get(value, faker_func()) for value in original_values]

    # Store mappings
    for original, fake_val in zip(original_values, fake_values):
        forward_mapping[original] = fake_val
        reverse_mapping[fake_val] = original

    df[column] = df[column].map(forward_mapping).fillna(df[column])
    return df

@time_it
def mask_sensitive_data(df, classified_columns):
    """Masks all detected sensitive columns using Faker."""
    for entity_type, columns in classified_columns.items():
        for column in columns:
            if column in df.columns:
                df = generate_fake_values(df, entity_type, column)

    return df

if __name__ == "__main__":
    load_mapping()
    input_file = "smaller_companies.csv"
    output_file = "anonymized.csv"

    df = pd.read_csv(input_file)
    classified_columns = load_and_analyze(input_file)

    df = mask_sensitive_data(df, classified_columns)
    df.to_csv(output_file, index=False)

    save_mapping()





Sensitive cols: ['name', 'industry', 'locality', 'country', 'linkedin url']


PII Columns: []
✅ Anonymized file saved as anonymized.csv


Execution time anonymization: 7.634835 seconds


<ipython-input-39-4f5d48a6ae44>:179: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: mapping.get(x, x))


✅ De-anonymized file saved as deanonymized.csv


Execution time de_anonymization: 0.533211 seconds


In [29]:
# Optimizing Thejus's logic
import os
import re
import json
import time
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime
from google.colab import files
from presidio_analyzer import AnalyzerEngine

# Initialize Faker & Presidio Analyzer
fake = Faker()
Faker.seed(42)
analyzer = AnalyzerEngine()

# Mapping file for storing forward/reverse mappings
MAPPING_FILE = "mappings.json"
forward_mapping = {}
reverse_mapping = {}

NUMERIC_PATTERN = re.compile(r"^[<>]?[\d,.%]+$")
SENSITIVE_HEADERS = {"phone", "mobile", "credit", "card", "id"}
EXCLUDED_ENTITIES = {"DATE_TIME"}  # Exclude unwanted entities


def time_it(func):
    """Decorator to measure execution time of functions."""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f'\n⏳ Execution time {func.__name__}: {end-start:.6f} seconds')
        return result
    return wrapper


def load_mapping():
    """Loads existing mapping from file, ensuring proper data handling."""
    global forward_mapping, reverse_mapping
    if os.path.exists(MAPPING_FILE):
        try:
            with open(MAPPING_FILE, "r") as f:
                data = json.load(f)
                forward_mapping = data.get("forward_mapping", {})
                reverse_mapping = data.get("reverse_mapping", {})
            print("📂 Existing mapping loaded.")
        except json.JSONDecodeError as e:
            print(f"⚠️ Error decoding JSON file: {e}")


def save_mapping():
    """Saves the mapping to a JSON file with string keys."""
    mapping_data = {
        "forward_mapping": {str(k): str(v) for k, v in forward_mapping.items()},
        "reverse_mapping": {str(k): str(v) for k, v in reverse_mapping.items()},
        "metadata": {
            "updated_at": datetime.now().isoformat(),
            "record_count": len(forward_mapping),
        },
    }
    with open(MAPPING_FILE, "w") as f:
        json.dump(mapping_data, f, indent=4)
    print("💾 Mapping saved successfully.")



@time_it
def analyze_column(df):
    """Uses Presidio to classify columns based on detected entity types."""
    entity_columns = {}
    for col in df.columns:
        col_lower = col.lower()
        if any(keyword in col_lower for keyword in SENSITIVE_HEADERS):
            entity_columns.setdefault("SENSITIVE", []).append(col)
            continue

        unique_values = df[col].dropna().astype(str).unique()[:10]
        entity_counts = {}

        for value in unique_values:
            if NUMERIC_PATTERN.match(value):
                continue  # Skip purely numeric values unless it's a phone/credit card

            results = analyzer.analyze(text=value, language="en")
            for result in results:
                if result.entity_type in EXCLUDED_ENTITIES:
                    continue  # Skip unwanted entities
                entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

        if entity_counts:
            predominant_entity = max(entity_counts, key=entity_counts.get)
            if predominant_entity not in entity_columns:
                entity_columns[predominant_entity] = []
            entity_columns[predominant_entity].append(col)

    return entity_columns


def generate_fake_values(column):
    """Generates consistent fake values for a column while preserving uniqueness."""
    col_name = column.name.lower()
    original_values = column.dropna().unique()

    if col_name in forward_mapping:
        mapping = forward_mapping[col_name]
    else:
        mapping = {}

    fake_values = []
    if re.search(r"\b(phone|mobile)\b", col_name, re.IGNORECASE):
        fake_values = [fake.phone_number() for _ in original_values]
    elif re.search(r"\b(credit|card)\b", col_name, re.IGNORECASE):
        fake_values = [fake.credit_card_number() for _ in original_values]
    elif re.search(r"\b(id|identifier)\b", col_name, re.IGNORECASE):
        fake_values = [fake.bothify(text="ID##########????").upper() for _ in original_values]
    elif re.search(r"\b(name|full[_\s]?name|first[_\s]?name|last[_\s]?name)\b", col_name, re.IGNORECASE):
        fake_values = [f'{fake.bothify(text="???#####")}--{fake.name()}' for _ in original_values]
    else:
        fake_values = [fake.bothify(text="????########").upper() for _ in original_values]


    mapping.update(dict(zip(original_values, fake_values)))
    forward_mapping[col_name] = mapping
    reverse_mapping[col_name] = {v: k for k, v in mapping.items()}

    return column.map(mapping).fillna(column)


@time_it
def mask_columnwise(df):
    """Replaces sensitive data only in columns detected as sensitive by Presidio."""
    sensitive_columns = analyze_column(df)

    for entity, cols in sensitive_columns.items():
        for column in cols:
            print(f"🔒 Masking sensitive column: {column} (Detected as {entity})")
            df[column] = generate_fake_values(df[column])

    return df


def restore_data(df):
    """Restores original values using stored mappings."""
    for column in df.columns:
        col_name = column.lower()
        if col_name in reverse_mapping:
            df[column] = df[column].astype(str).map(reverse_mapping[col_name]).fillna(df[column])
    return df


def check_if_files_match(original_file, restored_file):
    """Checks if the original and restored files match."""
    file_ext = os.path.splitext(original_file)[-1].lower()
    original_df = pd.read_csv(original_file) if file_ext == ".csv" else pd.read_excel(original_file)
    restored_df = pd.read_csv(restored_file) if file_ext == ".csv" else pd.read_excel(restored_file)
    test = original_df.equals(restored_df)
    print(f"📊 Are files identical? {test}")
    if not test:
        print("⚠️ Files are not identical!")
    else:
        print("✅ Files match perfectly!")


if __name__ == "__main__":
    load_mapping()
    # uploaded = files.upload()
    # input_file = list(uploaded.keys())[0].strip().replace(' ', '-')
    input_file = "ust.csv"

    anonymized_file = "anonymized.csv"
    restored_file = "restored.csv"

    df = pd.read_csv(input_file)
    df = mask_columnwise(df)  # Only masks detected sensitive columns
    df.to_csv(anonymized_file, index=False)
    save_mapping()

    df = restore_data(pd.read_csv(anonymized_file))
    df.to_csv(restored_file, index=False)

    check_if_files_match(input_file, restored_file)


In [31]:
if __name__ == "__main__":
    load_mapping()
    # uploaded = files.upload()
    # input_file = list(uploaded.keys())[0].strip().replace(' ', '-')
    input_file = "smaller_100k_companies.csv"

    anonymized_file = "anonymized.csv"
    restored_file = "restored.csv"

    df = pd.read_csv(input_file)
    df = mask_columnwise(df)  # Only masks detected sensitive columns
    df.to_csv(anonymized_file, index=False)
    save_mapping()

    df = restore_data(pd.read_csv(anonymized_file))
    df.to_csv(restored_file, index=False)

    check_if_files_match(input_file, restored_file)


⏳ Execution time analyze_column: 0.789924 seconds
🔒 Masking sensitive column: id (Detected as SENSITIVE)
🔒 Masking sensitive column: name (Detected as LOCATION)
🔒 Masking sensitive column: locality (Detected as LOCATION)
🔒 Masking sensitive column: country (Detected as LOCATION)
🔒 Masking sensitive column: domain (Detected as URL)
🔒 Masking sensitive column: linkedin url (Detected as URL)

⏳ Execution time mask_columnwise: 24.072139 seconds
💾 Mapping saved successfully.
📊 Are files identical? True
✅ Files match perfectly!


In [35]:
import os
import re
import json
import time
import pandas as pd
from faker import Faker
from datetime import datetime
from presidio_analyzer import AnalyzerEngine

# Initialize Faker & Presidio Analyzer
fake = Faker()
Faker.seed(42)
analyzer = AnalyzerEngine()

# Mapping file for storing forward/reverse mappings
MAPPING_FILE = "mappings.json"
forward_mapping = {}
reverse_mapping = {}

NUMERIC_PATTERN = re.compile(r"^[<>]?[\d,.%]+$")
SENSITIVE_HEADERS = {"phone", "mobile", "credit", "card", "id"}
EXCLUDED_ENTITIES = {"DATE_TIME"}  # Exclude unwanted entities

def load_mapping():
    """Loads existing mapping from file."""
    global forward_mapping, reverse_mapping
    if os.path.exists(MAPPING_FILE):
        try:
            with open(MAPPING_FILE, "r") as f:
                data = json.load(f)
                forward_mapping = data.get("forward_mapping", {})
                reverse_mapping = data.get("reverse_mapping", {})
            print("📂 Existing mapping loaded.")
        except json.JSONDecodeError as e:
            print(f"⚠️ Error decoding JSON file: {e}")

def save_mapping():
    """Saves the mapping to a JSON file."""
    mapping_data = {
        "forward_mapping": {str(k): str(v) for k, v in forward_mapping.items()},
        "reverse_mapping": {str(k): str(v) for k, v in reverse_mapping.items()},
        "metadata": {"updated_at": datetime.now().isoformat(), "record_count": len(forward_mapping)},
    }
    with open(MAPPING_FILE, "w") as f:
        json.dump(mapping_data, f, indent=4)
    print("💾 Mapping saved successfully.")

def analyze_column(df):
    """Uses Presidio to classify columns based on detected entity types."""
    entity_columns = {}
    for col in df.columns:
        col_lower = col.lower()
        if any(keyword in col_lower for keyword in SENSITIVE_HEADERS):
            entity_columns.setdefault("SENSITIVE", []).append(col)
            continue

        unique_values = df[col].dropna().astype(str).unique()[:10]
        entity_counts = {}

        for value in unique_values:
            if NUMERIC_PATTERN.match(value):
                continue  # Skip purely numeric values
            results = analyzer.analyze(text=value, language="en")
            for result in results:
                if result.entity_type in EXCLUDED_ENTITIES:
                    continue  # Skip unwanted entities
                entity_counts[result.entity_type] = entity_counts.get(result.entity_type, 0) + 1

        if entity_counts:
            predominant_entity = max(entity_counts, key=entity_counts.get)
            entity_columns.setdefault(predominant_entity, []).append(col)
    return entity_columns

def get_user_selected_columns(df):
    """Prompts the user to select additional sensitive columns."""
    print("\nColumns in the dataset:")
    for i, col in enumerate(df.columns):
        print(f"{i + 1}. {col}")

    selected_cols = input("Enter column numbers to mask (comma-separated): ")
    selected_cols = [df.columns[int(i) - 1] for i in selected_cols.split(",") if i.isdigit()]
    print("User-selected sensitive columns:", selected_cols)
    return selected_cols

def generate_fake_values(column):
    """Generates consistent fake values for a column while preserving uniqueness."""
    col_name = column.name.lower()
    original_values = column.dropna().unique()

    if col_name in forward_mapping:
        mapping = forward_mapping[col_name]
    else:
        mapping = {}

    fake_values = []
    if "phone" in col_name or "mobile" in col_name:
        fake_values = [fake.phone_number() for _ in original_values]
    elif "credit" in col_name or "card" in col_name:
        fake_values = [fake.credit_card_number() for _ in original_values]
    elif "id" in col_name:
        fake_values = [fake.bothify(text='??####').upper() for _ in original_values]
    elif "email" in col_name:
        fake_values = [fake.email() for _ in original_values]
    elif "url" in col_name:
        fake_values = [fake.url() for _ in original_values]
    else:
        fake_values = [fake.bothify(text="????########").upper() for _ in original_values]

    mapping.update(dict(zip(original_values, fake_values)))
    forward_mapping[col_name] = mapping
    reverse_mapping[col_name] = {v: k for k, v in mapping.items()}

    return column.map(mapping).fillna(column)

def mask_columnwise(df, user_selected_cols):
    """Replaces sensitive data in detected and user-selected columns."""
    sensitive_columns = analyze_column(df)
    for col in user_selected_cols:
        sensitive_columns.setdefault("MANUAL", []).append(col)

    for entity, cols in sensitive_columns.items():
        for column in cols:
            print(f"🔒 Masking column: {column} (Detected as {entity})")
            df[column] = generate_fake_values(df[column])
    return df

def restore_data(df):
    """Restores original values using stored mappings."""
    for column in df.columns:
        col_name = column.lower()
        if col_name in reverse_mapping:
            df[column] = df[column].astype(str).map(reverse_mapping[col_name]).fillna(df[column])
    return df

def check_if_files_match(original_file, restored_file):
    """Checks if the original and restored files match."""
    original_df = pd.read_csv(original_file)
    restored_df = pd.read_csv(restored_file)
    print(f"📊 Are files identical? {original_df.equals(restored_df)}")

if __name__ == "__main__":
    load_mapping()
    input_file = "sample.csv"
    anonymized_file = "anonymized.csv"
    restored_file = "restored.csv"

    df = pd.read_csv(input_file)
    user_selected_cols = get_user_selected_columns(df)
    df = mask_columnwise(df, user_selected_cols)
    df.to_csv(anonymized_file, index=False)
    save_mapping()

    df = restore_data(pd.read_csv(anonymized_file))
    df.to_csv(restored_file, index=False)
    check_if_files_match(input_file, restored_file)
())
